In [ ]:
import pygame
import random
import os

# 遊戲變數
FPS = 60
WIDTH = 800
HEIGHT = 500

BlueGray = (119, 136, 153)
Black = (0, 0, 0)
Red = (255, 0, 0)
Yellow = (255, 255, 0)
White = (255, 255, 255)

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("殺時小遊戲")
clock = pygame.time.Clock()

# 載入圖片
player_img = pygame.image.load(os.path.join("img", "satyr-Sheet 2-1.png")).convert()
background_img = pygame.image.load(os.path.join("img", "back.png")).convert()
stone_img = pygame.image.load(os.path.join("img", "stone.png")).convert()

# 載入音效
break_sound = pygame.mixer.Sound(os.path.join("mp3", "over.mp3"))
pygame.mixer.music.load(os.path.join("mp3", "back.mp3"))
pygame.mixer.music.play(-1)

game_over = False
score = 0

class Player(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.transform.scale(player_img, (36, 38))
        self.image.set_colorkey(White)
        pygame.display.set_icon(self.image)
        self.rect = self.image.get_rect()
        self.rect.x = 50
        self.rect.y = 450
        self.is_jumping = False
        self.jump_height = 0

    def jump(self):
        if not self.is_jumping:
            self.is_jumping = True
            self.jump_height = 100

    def update(self):
        key_pressed = pygame.key.get_pressed()
        if key_pressed[pygame.K_SPACE]:
            self.jump()

        if self.is_jumping:
            if self.jump_height > 0 and self.rect.y > 0:
                self.rect.y -= 10
                self.jump_height -= 10
            else:
                self.is_jumping = False
                self.jump_height = 0
        else:
            if self.rect.y < 450:
                self.rect.y += 10

class Obstacle(pygame.sprite.Sprite):
    def __init__(self, x, y, is_flipped):
        pygame.sprite.Sprite.__init__(self)
        obstacle_img = pygame.image.load(os.path.join("img", "stone.png")).convert()
        self.image = pygame.transform.scale(obstacle_img, (50, 300))
        self.image.set_colorkey(White)
        self.rect = self.image.get_rect()
        self.rect.x = x
        if is_flipped:
            self.image = pygame.transform.flip(self.image, False, True)
            self.rect.y = y - 300
        else:
            self.rect.y = y
        self.speedx = 4

    def update(self):
        self.rect.x -= self.speedx
        if self.rect.left < -60:
            self.rect.x = WIDTH + random.randrange(200, 400)
            self.rect.y = random.randrange(300, 450)
            self.speedx = 4

all_sprites = pygame.sprite.Group()
player = Player()
all_sprites.add(player)
obstacles = pygame.sprite.Group()

# 創建下方的障礙物
for i in range(1):
    obstacle = Obstacle(WIDTH + random.randrange(200, 400), random.randrange(300, 450), False)
    all_sprites.add(obstacle)
    obstacles.add(obstacle)

# 創建上方的障礙物
for i in range(1):
    obstacle = Obstacle(WIDTH + random.randrange(200, 400), random.randrange(0, 150), True)
    all_sprites.add(obstacle)
    obstacles.add(obstacle)

font = pygame.font.Font(None, 36)

running = True
while running:
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                if game_over:
                    game_over = False
                    score = 0
                    all_sprites.empty()
                    player = Player()
                    all_sprites.add(player)
                    obstacles.empty()

                    for i in range(1):
                        obstacle = Obstacle(WIDTH + random.randrange(200, 400), random.randrange(300, 450), False)
                        all_sprites.add(obstacle)
                        obstacles.add(obstacle)

                    for i in range(1):
                        obstacle = Obstacle(WIDTH + random.randrange(200, 400), random.randrange(0, 150), True)
                        all_sprites.add(obstacle)
                        obstacles.add(obstacle)

    if not game_over:
        all_sprites.update()
        hits = pygame.sprite.spritecollide(player, obstacles, True)

        all_sprites.draw(screen)
        if hits:
            game_over = True
            break_sound.play()

    background_img = pygame.transform.scale(background_img, (800, 500))
    screen.blit(background_img, (0, 0))
    all_sprites.draw(screen)

    # 創建新的障礙物
    for i in range(1):
        obstacle = Obstacle(WIDTH + random.randrange(200, 400), random.randrange(300, 450), False)
        all_sprites.add(obstacle)
        obstacles.add(obstacle)

    for i in range(1):
        obstacle = Obstacle(WIDTH + random.randrange(200, 400), random.randrange(0, 150), True)
        all_sprites.add(obstacle)
        obstacles.add(obstacle)

    if game_over:
        pygame.mixer.music.load(os.path.join("mp3", "back.mp3"))
        pygame.mixer.music.play(-1)
        text = font.render("Game Over! Press Space to Restart", True, Black)
        text_rect = text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text, text_rect)
    else:
        score += 1
        score_text = font.render(f"Score: {score}", True, Black)
        screen.blit(score_text, (10, 10))

    pygame.display.update()

pygame.quit()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
